In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
load_dotenv('my.env')
MODEL = os.getenv("MODEL")
API_KEY = os.getenv("API_KEY")
API_BASE = os.getenv("API_BASE")

llm = ChatOpenAI(
    model=MODEL,
    openai_api_key=API_KEY,
    openai_api_base=API_BASE
)

In [11]:
from pydantic import BaseModel, Field

class Profile(BaseModel):
    name: str = Field(description="Название по латински")
    size: float = Field(description="Типичный размер в микрометрах")
    weight: float = Field(description="Вес в микрограммах")
    shelf_life: int = Field(description="Срок жизни в днях")

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Profile)
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template=(
        "Ты учитель биологии. Твоя задача дать описание микроорганизмов. Ответь на вопрос в требуемом формате.\n"
        "{format_instructions}\n"
        "Вопрос: {user_question}\nОтвет:"
    ),
    input_variables=["user_question"],
    partial_variables={"format_instructions": format_instructions},
)

chain = prompt | llm | output_parser


In [18]:
try:
    result = chain.invoke({"user_question": "Расскажи о коловратке"})
    print(result.model_dump_json(indent=4))
except:
    print({"error": "Invalid response format"})

{
    "name": "Rotifera",
    "size": 0.1,
    "weight": 0.001,
    "shelf_life": 30
}


In [26]:
from langchain_core.exceptions import OutputParserException
import json
try:
    result = chain.invoke({"user_question": "Расскажи о небе"})
    print(result.model_dump_json(indent=4))
except OutputParserException as m:
    error_data = {  "error": "Invalid response format",
                    "message": str(m)}
    print(json.dumps(error_data, indent=4))

{
    "error": "Invalid response format",
    "message": "Failed to parse Profile from completion {\"name\": \"N/A\", \"size\": null, \"weight\": null, \"shelf_life\": null}. Got: 3 validation errors for Profile\nsize\n  Input should be a valid number [type=float_type, input_value=None, input_type=NoneType]\n    For further information visit https://errors.pydantic.dev/2.11/v/float_type\nweight\n  Input should be a valid number [type=float_type, input_value=None, input_type=NoneType]\n    For further information visit https://errors.pydantic.dev/2.11/v/float_type\nshelf_life\n  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]\n    For further information visit https://errors.pydantic.dev/2.11/v/int_type\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE "
}
